In [2]:
import pandas as pd
from matplotlib import pyplot as plt
#!pip install pivottablejs
import seaborn as sns

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
lpop = pd.read_stata('ARG_2021_LAPOP_AmericasBarometer_v1.2_w.dta',convert_categoricals=False)
gpt = pd.read_csv('respuestas_gpt35_turbo')
cohere = pd.read_csv('respuestas_cohere.csv')
bard = pd.read_csv('respuestas_bard.csv')
categorias = pd.read_csv('categorización_preguntas.csv')

In [3]:
categorias.head()

,preguntas,encuesta,categorias
0,A4- En su opinión ¿cuál es el problema más gra...,LPOP,opinion
1,JC13- Alguna gente dice que en ciertas circuns...,LPOP,democracia
2,JC13COVID- Alguna gente dice que en ciertas ci...,LPOP,democracia
3,JC15A- ¿Cree usted que cuando el país enfrenta...,LPOP,excesos del poder ejecutvio - presidencialismo
4,JCCOV1- ¿Cree usted que cuando hay una emergen...,LPOP,excesos del poder ejecutvio - presidencialismo


In [4]:
categorias['pregunta_codigo'] = categorias['preguntas'].apply(lambda x: x.split('-')[0])
categorias.head()

,preguntas,encuesta,categorias,pregunta_codigo
0,A4- En su opinión ¿cuál es el problema más gra...,LPOP,opinion,A4
1,JC13- Alguna gente dice que en ciertas circuns...,LPOP,democracia,JC13
2,JC13COVID- Alguna gente dice que en ciertas ci...,LPOP,democracia,JC13COVID
3,JC15A- ¿Cree usted que cuando el país enfrenta...,LPOP,excesos del poder ejecutvio - presidencialismo,JC15A
4,JCCOV1- ¿Cree usted que cuando hay una emergen...,LPOP,excesos del poder ejecutvio - presidencialismo,JCCOV1


In [5]:
categorias['categorias'].value_counts()

independencia de poderes: poder ejecutivo vs corte    8
democracia                                            6
opinion                                               5
confianza en las insitituciones                       5
relaciones internacionales                            5
economia                                              5
excesos del poder ejecutvio - presidencialismo        4
justicia                                              4
ingresos                                              2
derechos sociales                                     1
vivienda                                              1
desastres naturales                                   1
Name: categorias, dtype: int64

In [52]:
import pickle
with open(r"respuestas.pickle", "rb") as input_file:
    respuestas = pickle.load(input_file)
with open(r"preguntas_seleccionadas.pickle", "rb") as input_file:
    preguntas_seleccionadas = pickle.load(input_file)

In [103]:
preguntas_seleccionadas = list(categorias[(categorias['categorias']!='opinion') & (categorias['categorias']!='desastres naturales') ]['pregunta_codigo'].values)
preguntas_seleccionadas[1:20]

['JC13COVID',
 'JC15A',
 'JCCOV1',
 'JCCOV2',
 'COUNTFAIR1',
 'COUNTFAIR2',
 'COUNTFAIR3',
 'COUNTFAIR4',
 'WVSI2',
 'WVSI3',
 'CHM1BN',
 'CHM2BN',
 'CSES6N',
 'B2',
 'B3',
 'B6',
 'DST1BN',
 'ING4',
 'CCCH1']

In [16]:
preguntas_seleccionadas.keys()

dict_keys(['A4', 'JC13', 'JC13COVID', 'JC15A', 'JCCOV1', 'JCCOV2', 'COUNTFAIR1', 'COUNTFAIR2', 'COUNTFAIR3', 'COUNTFAIR4', 'WVSI2', 'WVSI3', 'CHM1BN', 'CHM2BN', 'CSES6N', 'B2', 'B3', 'B6', 'DST1BN', 'DRK1C', 'M1', 'ING4', 'ANESTG', 'MIL10A', 'MIL10E', 'CCCH1', 'CCUS1', 'CCCH3', 'CCUS3', 'CTAWARE', 'CTSPECIFIC1', 'CTDIFFUSE1', 'CTDIFFUSE2', 'OVQ11', 'OVQ21', 'OVQ12', 'OVQ22', 'OVQ13', 'OVQ23', 'OVQ14', 'OVQ24', 'TRADE5A', 'TRADE1A', 'TRADE2AA', 'TRADE2BB', 'TRADE2DA', 'TRADE3A'])

In [104]:
import math
pregs = []
def calculate_distance(responde,llm, preguntas):    
    result = 0
    num_pregs = 0
    for pregunta in list(preguntas):
        if((pregunta.lower() in responde.keys()) and (not math.isnan(responde[pregunta.lower()])) and (pregunta in list(llm['Pregunta'])) 
           and(llm[llm['Pregunta']== pregunta]['Respuesta'].values[0] < 10) ):
            
            result = result + (abs(responde[pregunta.lower()]-llm[llm['Pregunta']== pregunta]['Respuesta'].values[0]))
            num_pregs = num_pregs + 1
            pregs.append(pregunta)
    if(num_pregs == 0):
        result = -1
        norm_result = -1
    else:
        norm_result = result/num_pregs
    
    return(result, num_pregs, norm_result)
calculate_distance(lpop.loc[0],bard,preguntas_seleccionadas)

(24.0, 13, 1.8461538461538463)

In [118]:
for valor in lpop.columns:
    if valor.startswith('trade'):
        print(valor)

## ¿Porque no estan las preguntas de economía?

### En  el cuestionario dice "[NOTA DE PROGRAMACIÓN: BLOQUEAR TODOS LOS EXPERIMENTOS TRADE* POR GÉNERO]" ¿ Que significa??

In [4]:
# Crea un StataReader para leer los metadatos
reader = pd.io.stata.StataReader('ARG_2021_LAPOP_AmericasBarometer_v1.2_w.dta')

In [25]:
list(reader.value_labels()['CTAWARE_esp'.lower()].values())

AttributeError: 'list' object has no attribute 'str'

In [33]:
'comercio' in 'Apoya restringir el comercio'

True

In [31]:
for preg in reader.value_labels().keys():
    lowercase_list = [item.lower() for item in list(reader.value_labels()[preg].values())]
    if 'comercio' in lowercase_list:
        print(preg)

In [34]:
for preg in reader.value_labels().keys():
    
    if preg.lower().startswith('trade'):
        print(valor)

In [ ]:
Influencia positiva

In [ ]:
lpop = lpop.drop('a4', axis=1)

In [105]:
lpop['bard_results'] = lpop.apply(lambda x: calculate_distance(x, bard,preguntas_seleccionadas), axis=1)
lpop[['bard_distance', 'bard_num_preg','bard_distance_norm']] = pd.DataFrame(lpop['bard_results'].tolist(), index=lpop.index)

In [59]:
lpop['gpt_results'] = lpop.apply(lambda x: calculate_distance(x, gpt,preguntas_seleccionadas), axis=1)
lpop[['gpt_distance', 'gpt_num_preg','gpt_distance_norm']] = pd.DataFrame(lpop['gpt_results'].tolist(), index=lpop.index)

In [63]:
lpop['cohere_results'] = lpop.apply(lambda x: calculate_distance(x, cohere,preguntas_seleccionadas), axis=1)
lpop[['cohere_distance', 'cohere_num_preg','cohere_distance_norm']] = pd.DataFrame(lpop['cohere_results'].tolist(), index=lpop.index)

In [106]:
lpop[lpop['bard_num_preg'] >5].to_csv('lpop_bard_distances_nopinion.csv')

In [101]:
lpop[lpop['bard_num_preg'] >5]

,idnum,uniq_id,pais,year,wave,nationality,estratopri,strata,upm,prov1t,...,bard_num_preg,bard_distance_norm,gpt_results,gpt_distance,gpt_num_preg,gpt_distance_norm,cohere_results,cohere_distance,cohere_num_preg,cohere_distance_norm
0,236570.0,2.117024e+10,17,2021,2021,17,1702.0,1700002021,171360621,1700021.0,...,13,1.615385,"(11.0, 10, 1.1)",11.0,10,1.100000,"(5.0, 7, 0.7142857142857143)",5.0,7,0.714286
2,42281.0,2.117004e+10,17,2021,2021,17,1708.0,1700002021,171108421,1700001.0,...,14,0.785714,"(3.0, 10, 0.3)",3.0,10,0.300000,"(6.0, 9, 0.6666666666666666)",6.0,9,0.666667
6,206475.0,2.117021e+10,17,2021,2021,17,1708.0,1700002021,171627621,1700001.0,...,14,1.214286,"(8.0, 10, 0.8)",8.0,10,0.800000,"(4.0, 8, 0.5)",4.0,8,0.500000
7,382426.0,2.117038e+10,17,2021,2021,17,1706.0,1700002021,172519121,1700020.0,...,14,0.928571,"(4.0, 11, 0.36363636363636365)",4.0,11,0.363636,"(4.0, 7, 0.5714285714285714)",4.0,7,0.571429
8,272255.0,2.117027e+10,17,2021,2021,17,1702.0,1700002021,171481921,1700006.0,...,14,0.500000,"(6.0, 10, 0.6)",6.0,10,0.600000,"(2.0, 9, 0.2222222222222222)",2.0,9,0.222222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3003,417363.0,2.117042e+10,17,2021,2021,17,1708.0,1700002021,171892221,1700001.0,...,14,1.000000,"(6.0, 9, 0.6666666666666666)",6.0,9,0.666667,"(8.0, 8, 1.0)",8.0,8,1.000000
3004,302005.0,2.117030e+10,17,2021,2021,17,1708.0,1700002021,172302321,1700001.0,...,14,0.785714,"(3.0, 11, 0.2727272727272727)",3.0,11,0.272727,"(4.0, 8, 0.5)",4.0,8,0.500000
3005,423170.0,2.117042e+10,17,2021,2021,17,1708.0,1700002021,171867521,1700001.0,...,14,1.071429,"(8.0, 9, 0.8888888888888888)",8.0,9,0.888889,"(8.0, 9, 0.8888888888888888)",8.0,9,0.888889
3007,28502.0,2.117003e+10,17,2021,2021,17,1708.0,1700002021,172015421,1700001.0,...,14,0.714286,"(6.0, 12, 0.5)",6.0,12,0.500000,"(2.0, 8, 0.25)",2.0,8,0.250000


In [55]:
from collections import Counter

counter = Counter(pregs)

print(counter)

Counter({'ING4': 2984, 'ANESTG': 2981, 'CSES6N': 2832, 'B2': 1461, 'B3': 1457, 'CTAWARE': 1450, 'M1': 1447, 'B6': 1444, 'DRK1C': 1438, 'DST1BN': 1433, 'CTDIFFUSE2': 1409, 'JC15A': 1371, 'MIL10E': 1055, 'JCCOV2': 744, 'JC13': 729, 'MIL10A': 719, 'COUNTFAIR4': 710, 'CCUS1': 710, 'COUNTFAIR1': 687, 'COUNTFAIR3': 681, 'CCCH1': 679, 'JC13COVID': 670, 'COUNTFAIR2': 666, 'JCCOV1': 664})


In [56]:
pregs_bard = {'ING4': 2984, 'ANESTG': 2981, 'CSES6N': 2832, 'B2': 1461, 'B3': 1457, 'CTAWARE': 1450, 'M1': 1447, 'B6': 1444, 'DRK1C': 1438, 'DST1BN': 1433, 'CTDIFFUSE2': 1409, 'JC15A': 1371, 'MIL10E': 1055, 'JCCOV2': 744, 'JC13': 729, 'MIL10A': 719, 'COUNTFAIR4': 710, 'CCUS1': 710, 'COUNTFAIR1': 687, 'COUNTFAIR3': 681, 'CCCH1': 679, 'JC13COVID': 670, 'COUNTFAIR2': 666, 'JCCOV1': 664}

In [60]:
from collections import Counter

counter = Counter(pregs)

print(counter)

Counter({'CSES6N': 2832, 'DRK1C': 1438, 'DST1BN': 1433, 'CTDIFFUSE2': 1409, 'JC15A': 1371, 'JCCOV2': 744, 'JC13': 729, 'WVSI3': 715, 'COUNTFAIR1': 687, 'COUNTFAIR3': 681, 'CCCH1': 679, 'JC13COVID': 670, 'COUNTFAIR2': 666, 'CCUS3': 642, 'CCCH3': 604, 'B2': 1, 'B3': 1, 'B6': 1, 'M1': 1, 'ING4': 1, 'ANESTG': 1, 'MIL10E': 1, 'CTAWARE': 1})


In [61]:
pregs_gpt = {'CSES6N': 2832, 'DRK1C': 1438, 'DST1BN': 1433, 'CTDIFFUSE2': 1409, 'JC15A': 1371, 'JCCOV2': 744, 'JC13': 729, 'WVSI3': 715, 'COUNTFAIR1': 687, 'COUNTFAIR3': 681, 'CCCH1': 679, 'JC13COVID': 670, 'COUNTFAIR2': 666, 'CCUS3': 642, 'CCCH3': 604, 'B2': 1, 'B3': 1, 'B6': 1, 'M1': 1, 'ING4': 1, 'ANESTG': 1, 'MIL10E': 1, 'CTAWARE': 1}

In [64]:
from collections import Counter

counter = Counter(pregs)

print(counter)

Counter({'CTDIFFUSE2': 1409, 'JC15A': 1371, 'JCCOV2': 744, 'JC13': 729, 'WVSI3': 715, 'COUNTFAIR4': 710, 'CCUS1': 710, 'WVSI2': 694, 'COUNTFAIR1': 687, 'COUNTFAIR3': 681, 'CCCH1': 679, 'JC13COVID': 670, 'JCCOV1': 664, 'CHM1BN': 643, 'CHM2BN': 615, 'COUNTFAIR2': 1, 'CSES6N': 1, 'B2': 1, 'B3': 1, 'B6': 1, 'DST1BN': 1, 'DRK1C': 1, 'M1': 1, 'ING4': 1, 'ANESTG': 1, 'MIL10E': 1, 'CTAWARE': 1})


In [65]:
cohere_gpt = {'CTDIFFUSE2': 1409, 'JC15A': 1371, 'JCCOV2': 744, 'JC13': 729, 'WVSI3': 715, 'COUNTFAIR4': 710, 'CCUS1': 710, 'WVSI2': 694, 'COUNTFAIR1': 687, 'COUNTFAIR3': 681, 'CCCH1': 679, 'JC13COVID': 670, 'JCCOV1': 664, 'CHM1BN': 643, 'CHM2BN': 615, 'COUNTFAIR2': 1, 'CSES6N': 1, 'B2': 1, 'B3': 1, 'B6': 1, 'DST1BN': 1, 'DRK1C': 1, 'M1': 1, 'ING4': 1, 'ANESTG': 1, 'MIL10E': 1, 'CTAWARE': 1}

In [67]:
'CTDIFFUSE2' in cohere_gpt

True

In [66]:
categorias['cont_bard'] = 0

In [71]:
categorias['cont_bard'] = categorias['pregunta_codigo'].apply(lambda x: pregs_bard[x] if x in pregs_bard else 0)

In [80]:
categorias['cont_gpt'] = categorias['pregunta_codigo'].apply(lambda x: pregs_gpt[x] if x in pregs_gpt else 0)

In [81]:
categorias['cont_cohere'] = categorias['pregunta_codigo'].apply(lambda x: cohere_gpt[x] if x in cohere_gpt else 0)

In [84]:
categorias.groupby('categorias')['cont_bard'].sum().sort_values(ascending=False)

categorias
opinion                                               6202
democracia                                            5827
excesos del poder ejecutvio - presidencialismo        5611
confianza en las insitituciones                       4205
justicia                                              2859
derechos sociales                                     1457
desastres naturales                                   1438
vivienda                                              1433
relaciones internacionales                            1389
economia                                                 0
independencia de poderes: poder ejecutivo vs corte       0
ingresos                                                 0
Name: cont_bard, dtype: int64

In [85]:
categorias.groupby('categorias')['cont_gpt'].sum().sort_values(ascending=False)

categorias
excesos del poder ejecutvio - presidencialismo        4947
democracia                                            2116
confianza en las insitituciones                       2035
relaciones internacionales                            1925
desastres naturales                                   1438
vivienda                                              1433
justicia                                              1410
opinion                                                  3
derechos sociales                                        1
economia                                                 0
independencia de poderes: poder ejecutivo vs corte       0
ingresos                                                 0
Name: cont_gpt, dtype: int64

In [86]:
categorias.groupby('categorias')['cont_cohere'].sum().sort_values(ascending=False)

categorias
democracia                                            2810
excesos del poder ejecutvio - presidencialismo        2780
confianza en las insitituciones                       2080
justicia                                              1410
relaciones internacionales                            1389
ingresos                                              1258
opinion                                                  3
derechos sociales                                        1
desastres naturales                                      1
vivienda                                                 1
economia                                                 0
independencia de poderes: poder ejecutivo vs corte       0
Name: cont_cohere, dtype: int64